In [1]:
#Allow Notebook to Import from Comp_Chem_Package
import sys
if("win" in sys.platform):
    sys.path.append("..\\..\\Backend")
else: 
    sys.path.append("../../Backend")
    
from nistScraper import getDiatomicConstants 
from morsePotential import morsePotential
from howPotential import howPotential
from wavefunction import wavefunction
from operators import HOperator, VOperator, TOperator
from schrodinger import schrod
from basisSet import basisSet
from compChemGlobal import *
from how import how
from rkr import rkr

In [4]:
from diatomicConstants import buildDiatomicConstants

scaleFactor = 570

diatomicSymbol = widgets.Dropdown(
    options = ["CO", "HCl", "HF", "O2", "OH", "HF", "N2", "NO"], 
    value = "CO"
)
parameterOptions = ["Normal", "Times 2", "Divide 2"]
#u = widgets.Dropdown(
#    options = parameterOptions, 
#    description = "μ", 
#    value = "Normal"
#)
#w = widgets.Dropdown(
#    options = parameterOptions, 
#    description = "ω", 
#    value = "Normal"
#)
# comparison = widgets.Dropdown(
#     options = ["Harmonic", "Anharmonic"],
#     description = "Comparison", 
#     value = "Harmonic"
# )
firstState = widgets.Dropdown(
    options = ["ground"], 
    description = "1st State"
)
secondState = widgets.Dropdown(
   options = [], 
   description = "2nd State"
)
diatomic = widgets.HBox([widgets.Label("Diatomic Symbol"), diatomicSymbol, firstState, secondState])


calculate = startButton = widgets.ToggleButton(
    value = False, 
    description = "Update Solution",
    button_style = "info"
)

info = widgets.HTML("")
graph = widgets.VBox()
out = widgets.Output()
    
def inform(value):
    info.value = value
    out.clear_output()
    with out: 
        display(info)

#-------------------------------------------------------------------------------------

# def buildSolution(dc, name = "", size = 25, color = None):
#     pes =  howPotential(dc)
#     pes.color = color
#     basis = basisSet(dc, how, size)
#     sol = schrod()
#     sol.graphTitle = basis.diatomicConstants["name"] + " Harmonic Oscillator Schrödinger Solution"
#     for index, bf in enumerate(basis):
#         #eigen value analytical equation derived from hyperphysics website for harmonic osciallator
        
#         ev = (index + 0.5) * dc["w"]
#         evec = [1 if i == index else 0 for i in range(basis.size)]
#         wf = wavefunction(evec, ev, basis, index).scale(scaleFactor)
#         wf2 = wavefunction(evec, ev, basis, index, squared=True).scale(scaleFactor)
#         wf.setGraphVariables(yEqualsCutoff = ev, color = color)
#         wf2.setGraphVariables(yEqualsCutoff = ev, color = color)
        
#         sol.addGraphableObject(wf)
#         sol.addGraphableObject(wf2)
        
#     sol.addGraphableObject(pes)
#     sol.scaleFactor = scaleFactor
#     sol.start = dc["re"] - 0.4
#     sol.end = dc["re"] + 0.4
#     sol.pesLocations.append(len(sol.graphableObjects) - 1)
    
#     return sol
 
#-------------------------------------------------------------------------------------

def buildSolution(dc, color, size=10):
    basis = basisSet(dc, basisFunctionClass=how, size=size)
    
    pesMethod = rkr(dc)
    pesMethod.compute()
        
    mp = morsePotential(dc).fit(pesMethod.getData())
    
    H = TOperator(basis) + VOperator(basis, mp)
    
    sol = schrod(color=color, linearModification=dc["T"])
    sol.solve(H, basis, mp)
    
    return sol
    
#-------------------------------------------------------------------------------------

def howPESUpdate():
    
    states = getDiatomicConstants(diatomicSymbol.value, returnStates=True)[::-1]
    print(states)
    print(firstState.options)
    print()
    print(states == list(firstState.options))
    if(list(firstState.options) != states):
        firstState.options = states
        secondState.options = states
        
        firstState.value = states[-1]
        secondState.value = states[-2]
    
    dc1 = getDiatomicConstants(diatomicSymbol.value, firstState.value)
    dc2 = getDiatomicConstants(diatomicSymbol.value, secondState.value)
    print(dc2)
    if(dc1 == False):
        info.value = "'" + diatomicSymbol.value + "' is not a valid diatomic molecule." 
        return
    info.value = "Calculating solution for " + diatomicSymbol.value + "..." 
    
    sol = buildSolution(dc1, "red")
    sol.combineSolutions(buildSolution(dc2, "blue"), " a 3Πr")
    #sol = buildSolution(dc, color = "red")
    #name = ""
    #update diatomic constants
#     if(comparison.value == "Harmonic"):
#         for widget, index in [(u, "u"), (w, "w")]:
#             if(widget.value == "Times 2"):
#                 dc[index] *= 2
#                 name += ", " + widget.description + " " + widget.value
#             elif(widget.value == "Divide 2"):
#                 dc[index] /= 2
#                 name += ", " + widget.description + " " + widget.value
                
#         if(name != ""):
#             sol.combineSolutions(buildSolution(dc, name, color = "blue"), name)
#     else: 
#         sol2 = schrod()
#         sol2.load(dc["name"], "Harmonic Oscillator", 100, "Extended Rydberg Potential", "RKR", scaleFactor = scaleFactor, color = "blue")
#         sol.combineSolutions(sol2, " Anharmonic")
    
    endPoints = [dc1["re"] - 1, dc1["re"] + 1]
    sol.start, sol.end = endPoints[0], endPoints[1]
    graph.children = ( (sol.graph(getGraph = True), diatomic, calculate, info) )
    graph.children[0].children[0].update_xaxes(range=endPoints).update_yaxes(range=[min((dc1["T"], dc2["T"])),sol.graphableObjects[1].energy + 5000 + dc2["T"]])
    
    out.clear_output()
    with out: 
        display(graph)
    
    info.value = "Calculation Completed"

updateWrapper = lambda change :  howPESUpdate()

calculate.observe(updateWrapper, names="value")

display(out)
inform("Building Graph...")

howPESUpdate()  

Output()

['(ndσ,π)', '(npσ,π)', '(nsσ)', '(npσ)', 'D3', 'U', 'D2', 'S2', 'T', 'R', 'S1', '(nsσ)', '(npσ,π)', 'Q', 'O2 (1Π)', 'P', 'O1 (1Π)', 'N', 'npπ', 'npσ', 'nsσ', "I' (5sσ)", 'Z 1Σ+', 'H (1Π)', "H' 1Π", 'L 1Π 4pπ', "L' (1Π)", 'K 1Σ+ 4pσ', 'W 1Π', "W' 1Π", 'J 1Σ+ 4sσ', 'G 1Π(3dπ)', "G' 1Π", 'h (3Σ+4sσ)', 'Y 1Σ+', 'F 1Σ+(3dσ)', 'V 1Π', 'g 3Σ+', 'E 1Π 3pπ', 'E  X  V', 'c 3Π 3pπ', 'c  X V', 'C 1Σ+ 3pσ', 'C  X   V', 'j (3Σ+3pσ)', '', 'k', 'B 1Σ+ 3sσ', 'B  X   V', 'b 3Σ+ 3sσ', 'b  X  V', 'f (3Σ+)', 'D 1Δ', 'I 1Σ-', 'A 1Π', 'e 3Σ-', 'e  X T', 'd 3Δi', 'd  X R', "a' 3Σ+", "a'  X  R", 'a 3Πr', 'X 1Σ+']
('ground',)

False
{'name': 'CO', 'state': 'a 3Πr', 'T': 48686.7, 'w': 1743.41, 'wx': 14.36, 'wy': 0, 'wz': 0, 'B': 1.69124, 'a': 0.01904, 'y': 0, 'D': 6.36e-06, 're': 1.20574, 'u': 6.8562086380025615, 'De': None}
